<a href="https://colab.research.google.com/github/olgaszatkowska/non-linear-systems-identyfication/blob/main/F_16_ground_vibration_test_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from enum import Enum
from dataclasses import dataclass
from numpy.typing import NDArray
import numpy as np

class StimulationType(Enum):
  SINE_SWEEP = "sine_sweep"
  MULTISINE_FULL_FREQUENCY_GRID = "multisine_full_frequency_grid"
  MULTISINE_RANDOM_FREQUENCY_GRID = "multisine_random_frequency_grif"


@dataclass
class DataSample:
  force: float
  voltage: float
  acceleration_1: float
  acceleration_2: float
  acceleration_3: float
  fs: int|None
  simulaton_type: StimulationType

  @property
  def X(self) -> NDArray:
    return np.array([
        self.force,
        self.voltage,
    ])


  @property
  def y(self) -> NDArray:
    return np.array([
        self.acceleration_1,
        self.acceleration_2,
        self.acceleration_3
    ])


In [ ]:
def determine_stimulation_type(file_name: str) -> StimulationType:
  if ("SineSw") in file_name:
    return StimulationType.SINE_SWEEP
  if ("FullMSine") in file_name:
    return SimulationType.MULTISINE_FULL_FREQUENCY_GRID
  if ("SpecialOddMSine") in file_name:
    return SimulationType.MULTISINE_RANDOM_FREQUENCY_GRID

  raise Exception("Faile to determine simulation type")

In [ ]:
DIRECTORY = '/content/drive/My Drive/'
FOLDER_PATH = DIRECTORY + 'f-16_ground_vibration_test'

In [ ]:
import os
import csv

def csv_line_to_data_sample(line: list[str], stimulation_type: StimulationType) -> DataSample:
  try:
    if len(line) == 0:
      raise ValueError

    data_as_floats = [float(measurment) for measurment in line[:6]]
  except ValueError:
    raise

  return DataSample(*data_as_floats, stimulation_type)

def csv_file_to_data_samples(stimulation_type: StimulationType = None) -> list[DataSample]:
  data_samples = []

  for data_set_filename in os.listdir(FOLDER_PATH):
    filepath = os.path.join(FOLDER_PATH, data_set_filename)
    file_stimulation_type = determine_stimulation_type(data_set_filename)

    if (stimulation_type != None and stimulation_type != file_stimulation_type):
      pass

    with open(filepath, newline='') as data_set_file:
      reader = csv.reader(data_set_file)

      for row in reader:
        try:
          data_sample = csv_line_to_data_sample(row, stimulation_type)
          data_samples.append(data_sample)
        except ValueError:
          pass

  return data_samples

data_samples = csv_file_to_data_samples()

In [ ]:
len(data_samples)

147473

In [ ]:
from sklearn.model_selection import train_test_split
from numpy.typing import NDArray


def get_training_validation_and_testing_sets(data_samples: list[DataSample], window_size: int, stimulation_type: StimulationType = None) -> NDArray:
  X, y = [], []

  for i in range(0, len(data_samples), window_size):
    window = data_samples[i:i+window_size]
    X_window = np.array([sample.X for sample in window])
    y_window = np.array([sample.y for sample in window])
    X.append(X_window)
    y.append(y_window)

  X_train_val, X_test, y_train_val, y_test = train_test_split(np.asarray(X, dtype="object"), np.asarray(y, dtype="object"), test_size=0.2, random_state=42)

  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

  return [
      X_train,
      y_train,
      X_val,
      y_val,
      X_test,
      y_test,
  ]

In [ ]:
sets = get_training_validation_and_testing_sets(data_samples, 15, StimulationType.SINE_SWEEP)
X_train,y_train,X_val,y_val,X_test, y_test, = sets

In [ ]:
X_train[0]

array([[-30.2464,  62.5126],
       [-20.3824, 106.8808],
       [-11.0704, 112.744 ],
       [ -2.5735,  78.9595],
       [  4.704 ,  29.3849],
       [ 10.5553,  -1.2675],
       [ 15.1338,   9.2066],
       [ 18.1836,  54.3019],
       [ 17.8563, 101.8935],
       [ 10.7234, 115.2337],
       [ -5.8072,  77.7479],
       [-29.5226,   4.7418],
       [-51.009 , -66.085 ],
       [-56.193 , -99.2243],
       [-34.2517, -84.6282]])

In [ ]:
y_train[0]

array([[ -15.3005,   48.1422,  -41.6117],
       [   5.1462,   42.4999,  -41.8117],
       [   6.0308,   54.4885,  -37.1566],
       [ -10.4655,   72.5345,  -23.8776],
       [ -28.4185,   78.4518,   -1.8   ],
       [ -30.5534,   57.9408,   24.6613],
       [ -11.7665,    9.9175,   48.3306],
       [  16.3129,  -50.7975,   62.8988],
       [  33.6162,  -99.8142,   66.1458],
       [  27.1287, -115.2578,   60.8503],
       [   1.7372,  -88.5932,   52.7201],
       [ -22.571 ,  -28.7562,   46.3728],
       [ -25.5486,   41.276 ,   41.8916],
       [  -2.1908,   93.3255,   34.5302],
       [  32.9675,  105.3452,   18.079 ]])

9832